In [1]:
from sample_squad_data import *
# from sample_another_data1 import *
import pickle
import torch
import spacy

from transformers import BertTokenizer
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import copy


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\Admin\envs\machineLearning\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\Admin\envs\machineLearning\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree.

In [19]:
#GIAI ĐOẠN CHUẨN BỊ DATA CHO FINE TUNING
#CÁC ĐOẠN CODE NÀY ĐÃ ĐƯỢC CHẠY VÀ LƯU VÀO FILE PICKLE (chạy rất lâu)
#LẤY STYLE
styles_text = []
for i in loaded_list[0][0:20000]:
    styles_text.append(styles[i[0]])
    
#LẤY ANSER_TEXT   
ft_ans_text = []
for i in t[0:20000]:
    ft_ans_text.append(i['answer_text'])

#LẤY CÂU CHỨA ANSWER
ft_ans_sent = []
for i in t[0:20000]:
    ft_ans_sent.append(i['ans_sent'])

#LẤY RA CÁC CLUE
ft_clue = []
a = 0
for i in t[0:20000]:
    if a % 1000 == 0:
        print(a)
    iob = get_iob(i['ans_sent'], i['answer_text'], i['answer_start'])
    z = clue_text_ft(i['ans_sent'], i['answer_text'], iob, 7)
    ft_clue.append(z)
    a+=1
# 

In [ ]:
# fineturning T5

In [2]:
import pickle
import torch
import spacy
from transformers import BertTokenizer
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
    

In [18]:
# load data
with open('./Data/ft_style.pkl', 'rb') as file:
    styles_text= pickle.load(file)
with open('./Data/ft_ans_text.pkl', 'rb') as file:
    ft_ans_text= pickle.load(file)
with open('./Data/ft_ans_sent.pkl', 'rb') as file:
    ft_ans_sent= pickle.load(file)
with open('./Data/ft_clue.pkl', 'rb') as file:
    ft_clue= pickle.load(file)

In [3]:
with open('id_big.pkl', 'rb') as file:
    id_big = pickle.load(file)

In [11]:
id = [i for i in id_big  if (i[0] <20000)]

In [13]:
# loại bỏ những câu có độ dài quá lớn
styles_text_new = [item for idx, item in enumerate(styles_text) if [idx] not in id]
ft_ans_text_new = [item for idx, item in enumerate(ft_ans_text) if [idx] not in id]
ft_ans_sent_new = [item for idx, item in enumerate(ft_ans_sent) if [idx] not in id]

In [20]:
#tạo data frame
df = pd.DataFrame({
    'Answer': ft_ans_text_new,
    'Style': styles_text_new,
    'Source': ft_ans_sent_new
})
 
df['Output'] = 'answer: ' + df['Answer'] + ' style: ' + df['Style']
df.columns


Index(['Answer', 'Style', 'Source', 'Output'], dtype='object')

In [21]:
from datasets import Dataset
dataset = Dataset.from_pandas(df[['Source', 'Output']])

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration,Trainer, TrainingArguments

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

C:\Users\Admin\envs\machineLearning\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
# taoh mẫu cho dữ liệu đầu vào: 
def preprocess_function(examples):
    inputs = [f"get anwer style of: {s}" for s in examples['Source']]
    model_inputs = tokenizer(inputs, padding='max_length',max_length=88, truncation=True)
    
    # Tokenize output (clue + answer)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['Output'], padding='max_length',max_length=90, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = dataset.map(
    preprocess_function,
    batched=True,
)


Map:   0%|          | 0/19725 [00:00<?, ? examples/s]

C:\Users\Admin\envs\machineLearning\Lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [24]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
optimizer = torch.optim.Adam(
        params=model.parameters(), lr=0.001
    )

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
model = model.to(device)

In [27]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=0.001,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Sử dụng Trainer API để huấn luyện
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_train,
)

# Bắt đầu quá trình huấn luyện
trainer.train()

C:\Users\Admin\envs\machineLearning\Lib\site-packages\accelerate\accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.070700,0.049606
2,0.055500,0.036835
3,0.044300,0.029482


TrainOutput(global_step=9864, training_loss=0.06251982886522453, metrics={'train_runtime': 2501.8919, 'train_samples_per_second': 23.652, 'train_steps_per_second': 3.943, 'total_flos': 1376521283174400.0, 'train_loss': 0.06251982886522453, 'epoch': 3.0})

In [42]:
input_ids = tokenizer("get anwer style of:i have 2 brother", return_tensors="pt").input_ids.to(device)
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

answer: 2 style: how


C:\Users\Admin\envs\machineLearning\Lib\site-packages\transformers\generation\utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [40]:
# lưu lại mô hình t5
model.save_pretrained('T5_ans_clue_style')
tokenizer.save_pretrained('T5_ans_clue_style')

('T5_ans_clue_style\\tokenizer_config.json',
 'T5_ans_clue_style\\special_tokens_map.json',
 'T5_ans_clue_style\\spiece.model',
 'T5_ans_clue_style\\added_tokens.json')

In [3]:
from huggingface_hub import login

In [4]:
login()

In [5]:
# m = T5ForConditionalGeneration.from_pretrained('./T5/T5_ans_clue_style')
# t = T5Tokenizer.from_pretrained('./T5/T5_ans_clue_style')

In [6]:
# m.push_to_hub("T5_clue_style")
# t.push_to_hub("T5_clue_style")

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HF-Hao/T5_clue_style/commit/4c9db1dfb4ae25e6437d7ce267e5fea8f81d939a', commit_message='Upload tokenizer', commit_description='', oid='4c9db1dfb4ae25e6437d7ce267e5fea8f81d939a', pr_url=None, pr_revision=None, pr_num=None)

In [9]:
#load mô hình T5 để sử dụng
model_api = T5ForConditionalGeneration.from_pretrained('HF-Hao/T5_clue_style')
tokenizer_api = T5Tokenizer.from_pretrained('HF-Hao/T5_clue_style')
input_ids = tokenizer_api("get anwer style of: Beam Search is a popular technique in generating the best outputs from the model", return_tensors="pt").input_ids
outputs = model_api.generate(input_ids, num_beams=5, num_return_sequences=3)
print(tokenizer_api.decode(outputs[0], skip_special_tokens=True))

C:\Users\Admin\envs\machineLearning\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\Admin\envs\machineLearning\Lib\site-packages\transformers\generation\utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


answer: Beam Search style: what


In [3]:
#hàm để lấy clue text
def clue_text_ft(context, answer, answer_bio_ids, max_dependency_distance, processed_by_spacy=False):
    answer_start = answer_bio_ids.index('B')
    try:
        answer_end = list(reversed(answer_bio_ids)).index('I')
        answer_end = len(answer_bio_ids) - 1 - answer_end
    except:
        answer_end = answer_start
    if not processed_by_spacy:
        doc = NLP(context)
    else:
        doc = context

    doc_token_list = [token for token in doc]

    idx2token, idx2related, context_tokens = get_all_related(doc, doc_token_list)
    clue_flags = [0] * len(doc)
    for aid in range(answer_start, answer_end + 1):
        sort_related = idx2related[aid]
        for tk_id, path in sort_related:
            if (tk_id < answer_start or tk_id > answer_end) and len(path) <= max_dependency_distance:
                cur_clue = idx2token[tk_id]
                if cur_clue.pos_ not in ['ADP', 'DET', 'ADV', 'PUNCT', 'PART']:
                    clue_flags[tk_id] = 1
    clues = []
    i = 0
    while i < len(clue_flags):
        if clue_flags[i] == 0:
            i += 1
            continue
        j = i
        while j < len(clue_flags):
            if clue_flags[j] == 1:
                j += 1
            else:
                break
        clue_text = ' '.join(context_tokens[i:j])
        clues.append(clue_text)
        i = j
    clues = ', '.join(clues)
    return clues

In [21]:
def find_subtext(text, subtext):
    return text.find(subtext)


10

In [24]:
def create_input(sent, st, answ, styles, yes_no_ans, pos_tags_list, ner_tags_list):
    style_ids=[]
    pos_ids=[]
    ner_ids = []
    iob_tags = []
    is_clue = []
    style_ids.append(list(get_style_ids([st], styles, yes_no_ans)))
    p, n = get_pos_ner_ids(sent, pos_tags_list, ner_tags_list)
    pos_ids.append(p)
    ner_ids.append(n)
    ans_start= find_subtext(sent, answ)
    iob = get_iob(sent, answ, ans_start)
    iob_tags.append(iob)
    is_clue.append(get_clue_ids_new(sent, answ, iob, 7))
    return style_ids, pos_ids, ner_ids, iob_tags, is_clue  

SyntaxError: invalid syntax (980921373.py, line 1)

In [25]:
final = create_input('i have 2 brother','how', '2', styles, yes_no_ans, pos_tags_list, ner_tags_list)
final
#cho những đối tượng này qua model để đưa ra câu hỏi

([[6]],
 [[10, 15, 8, 7]],
 [[18, 18, 17, 18]],
 [['O', 'O', 'B', 'O']],
 [array([1, 1, 0, 1])])

In [ ]:
# tiền xử lý data cho giai đoạn train

In [9]:

with open('glove_model.pkl', 'rb') as f:
    GLOVE = pickle.load(f)
styles=['who', 'where', 'when', 'why', 'which', 'what', 'how','yes-no', 'other']
yes_no_ans = ['am', 'is', 'was', 'were', 'are', 'does', 'do', 'did'
              , 'have', 'had', 'has', 'could', 'can', 'shall', 'should', 'will', 'would', 'may', 'might']
pos_tags_list = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
ner_tags_list = ['PERSON', 'NORP', 'FAC', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL', 'O']


In [3]:
def get_clue_ids_new(context, answer, answer_bio_ids, max_dependency_distance, processed_by_spacy=False):
    answer_start = answer_bio_ids.index('B')
    try:
        answer_end = list(reversed(answer_bio_ids)).index('I')
        answer_end = len(answer_bio_ids) - 1 - answer_end
    except:
        answer_end = answer_start
    if not processed_by_spacy:
        doc = NLP(context)
    else:
        doc = context

    doc_token_list = [token for token in doc]
    # text_str = ' '.join([tk.text for tk in doc])

    idx2token, idx2related, context_tokens = get_all_related(doc, doc_token_list) #import
    clue_flags = [0] * len(doc)
    for aid in range(answer_start, answer_end + 1):
        sort_related = idx2related[aid]
        for tk_id, path in sort_related:
            if (tk_id < answer_start or tk_id > answer_end) and len(path) <= max_dependency_distance:
                cur_clue = idx2token[tk_id]
                if cur_clue.pos_ not in ['ADP', 'DET', 'ADV', 'PUNCT', 'PART']:
                    clue_flags[tk_id] = 1
    clues = []
    i = 0
    while i < len(clue_flags):
        if clue_flags[i] == 0:
            i += 1
            continue
        j = i
        while j < len(clue_flags):
            if clue_flags[j] == 1:
                j += 1
            else:
                break
        clue_text = ' '.join(context_tokens[i:j])
        clue_binary_ids = [0] * len(clue_flags)
        clue_binary_ids[i:j] = [1] * (j - i)
        clues.append({"clue_text": clue_text, "clue_binary_ids": clue_binary_ids})
        i = j
    ids = []
    for clue in clues:
        ids.append(clue['clue_binary_ids'])
    if len(ids)== 0:
        ids = np.zeros(len([token.text for token in NLP(context)]), dtype ='int')
    else:
        ids = np.array(ids)
        ids =np.sum(ids, axis=0)

    for i, id in enumerate(ids):
        if id>0:
            ids[i]=1
        else:
            ids[i]=0

    return ids

In [4]:
def get_pos_ner_ids(sentence, pos_tags_list, ner_tags_list):
    doc = NLP(sentence)
    pos_tags = [token.pos_ for token in doc]
    ner_tags = [token.ent_type_ if token.ent_type_ else 'O' for token in doc]
    pos_ids = [pos_tags_list.index(item) if item in pos_tags_list else pos_tags_list.index('X') for item in pos_tags]
    ner_ids = [ner_tags_list.index(item) for item in ner_tags]
    return pos_ids, ner_ids

In [3]:
t =get_squad_raw_examples('./Data/train.txt')
t[1]

Start get SQuAD raw examples ...
Time of get raw examples: 0:00:06.304222
Number of raw examples:  86635


{'question': 'What is in front of the Notre Dame Main Building?',
 'ans_sent': 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".',
 'answer_text': 'a copper statue of Christ',
 'answer_start': 60}

In [7]:
# hàm này trả về các thông tin như style ids, pos_ids, ner_ids, is_clue
def final_data(datas, styles, yes_no_ans, pos_tags_list, ner_tags_list, topN = 20):
    style_ids=[]
    pos_ids=[]
    ner_ids = []
    iob_tags = []
    is_clue = []
    
    for i, data in enumerate(datas[0:51000]):
        ans_sent = data['ans_sent']
        ques = data['question']
        ans_text = data['answer_text']
        ans_start = data['answer_start']
        style_ids.append(list(get_style_ids([ques], styles, yes_no_ans)))
        p, n = get_pos_ner_ids(ans_sent, pos_tags_list, ner_tags_list)
        pos_ids.append(p)
        ner_ids.append(n)
        iob = get_iob(ans_sent, ans_text, ans_start)
        iob_tags.append(iob)
        is_clue.append(get_clue_ids_new(ans_sent, ans_text, iob, 7))
    return style_ids, pos_ids, ner_ids, iob_tags, is_clue  

In [11]:
# final = final_data(t, styles, yes_no_ans, pos_tags_list, ner_tags_list)
#ĐÂY LÀ HÀM ĐỂ LẤY CÁC THÔNG TIN STYLE IDS, POS_TAG_IDS, NER_TAG_IDS, IS_CLUE
# !!!! CHẠY ĐOẠN CODE NÀY MẤT HƠN 1 TIẾNG

In [18]:
# with open('list_ids_data1.pkl', 'wb') as file:
#     pickle.dump(final, file)
#luu lại các giá trị data đã được xử lý
#ĐÃ ĐƯỢC LƯU
